## CS431/631 Data Intensive Distributed Analytics
### Winter 2021 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Hasnain Syed
* **ID:** 20565033

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://www.student.cs.uwaterloo.ca/~cs451/assignment1-431.html).

We first need to install Apache Spark. Run the next block to download and install Spark. It will take about a minute to finish the instalation.

In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located. The code in the following cell tells Python how to find this Spark installation. This code creates SparkContext (sc) for you. Do not change this block.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster, and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Next, let's test that your `SparkContext` has been set up properly by running some simple test code.   This code uses a single Spark job to estimate the value of Euler's number $e$. One way to calculate $e$ is to use the following serries by Jacob Bernoulli:

$p_n = 1 - \frac{1}{1!} + \frac{1}{2!} - \frac{1}{3!} + \cdots + \frac{(-1)^n}{n!} = \sum_{k = 0}^n \frac{(-1)^k}{k!}$

As n tends to infinity, $p_n$ approaches $1/e$.

In the following code,  `parallelize()` and `map()` are Spark *transformations*, and `reduce()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take several seconds, since a Spark job is being created and executed, and should print an estimate of $e$ when it finishes.   


In [ ]:
import math

n= 10000
inverse_e = sc.parallelize(range(0, n)).map(lambda x: ((-1)**x) * (1 / math.factorial(x))).reduce(lambda x,y:x+y)
e = 1 / inverse_e
print("e = ", e)

e =  2.718281828459044


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the example works.   What is the Spark job doing, and how is it used to estimate the value of $e$? How accurate is our estimate?

#### Your answer to Question 1:

The formula to calculate Euler's number is calculated by a summation of an equation where "n" goes to infinity. Of course, we cannot actually have a piece of code that will tend to infinity, but we can simulate the same effect by having a very large number. 

In the example given, we have chose to simulate this by having n=10000. First we take a list of integers from 0 to 9999, and parallelize it to a Spark RDD. Each of the integers are than mapped to the summation formula, which are then reduced by adding all of them up to get the value of 1/e. Then, 1/e is simply inversed to get the value of e.

The value of 1/e is very accruate to a certain number of digitis. This is because as we approach infinity, the very large n values have a very little affect on the value of 1/e because they result in a very large denominator, thus a very small value. The first few n values are very important to estimate the value of 1/e, which is then used to calculate e.

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Run the next block to download the text file (Shakespeare.txt) and the Python tokenizer module (simple_tokenize.py).

In [ ]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [ ]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
  
  lines = sc.textFile("Shakespeare.txt")
  distinct_tokens = lines.flatMap(lambda line: simple_tokenize(line)).map(lambda word: (word.lower().strip(), 1)).groupByKey().count()

  return distinct_tokens
  

print("Number of distinct tokens = " + str(count_distinct_tokens()))

Number of distinct tokens = 25975


---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [ ]:
from simple_tokenize import simple_tokenize
from itertools import permutations

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():

    lines = sc.textFile("Shakespeare.txt")
    distinct_pairs = lines.flatMap(lambda line: permutations(simple_tokenize(line),2)).filter(lambda x: x[0] != x[1]).map(lambda x: ((x[0], x[1]), 1)).groupByKey().count()

    return distinct_pairs

print("Number of distinct pairs = " + str(count_distinct_pairs()))

    

Number of distinct pairs = 1969760


---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [ ]:
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
    
    lines = sc.textFile("Shakespeare.txt")
    totalLines = lines.count()
    occurences = lines.flatMap(lambda line: set(simple_tokenize(line))).map(lambda word: (word.lower().strip(), 1)).reduceByKey(lambda x, y: x+y)
    top50 = occurences.map(lambda x: (x[1]/totalLines,(x[1], x[0]))).sortByKey(False).map(lambda x: (x[0], x[1][0], x[1][1])).take(50)

    return top50

print("Top 50 Token Probabilites : ") 
top_50_tokens_probabilities()


Top 50 Token Probabilites : 


[(0.2009178657172255, 24604, 'and'),
 (0.19843538192686472, 24300, 'the'),
 (0.1523542765682928, 18657, 'i'),
 (0.148924529226347, 18237, 'to'),
 (0.1357526662202551, 16624, 'of'),
 (0.10844534452628657, 13280, 'a'),
 (0.09959332995802643, 12196, 'you'),
 (0.09430988583840991, 11549, 'my'),
 (0.08667461497003054, 10614, 'in'),
 (0.08630714204053634, 10569, 'that'),
 (0.07150206601447026, 8756, 'is'),
 (0.06720671577193814, 8230, 'not'),
 (0.06167012363422561, 7552, 'with'),
 (0.06039621747864574, 7396, 'me'),
 (0.05982459292165477, 7326, 'for'),
 (0.05836286726877787, 7147, 'it'),
 (0.054402325695340446, 6662, 'be'),
 (0.05246696826667102, 6425, 'this'),
 (0.05228731483447386, 6403, 'his'),
 (0.050899083767495794, 6233, 'your'),
 (0.05067043394469941, 6205, 'but'),
 (0.0474938346208496, 5816, 'he'),
 (0.046889545803459144, 5742, 'have'),
 (0.04132028940534714, 5060, 'thou'),
 (0.04015254209606559, 4917, 'as'),
 (0.03952375508337552, 4840, 'him'),
 (0.03949109082297604, 4836, 'so'),
 (0

---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [ ]:
from simple_tokenize import simple_tokenize
from math import log
from itertools import permutations

#Global variables to broadcast occurences
lines = sc.textFile("Shakespeare.txt")
occurence = lines.flatMap(lambda line: set(simple_tokenize(line))).map(lambda word: (word.lower().strip(), 1)).reduceByKey(lambda x,y: x+y).collectAsMap()
broadcastedOcc = sc.broadcast(occurence)

#Helper function to return token occurences using broadcasted variable
def getTokenOccurence(token):
  return broadcastedOcc.value[token]

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)  
def PMI(threshold):
    lines = sc.textFile("Shakespeare.txt")
    totalLines = lines.count()
    coOccurences = lines.map(lambda x: set(simple_tokenize(x))).map(lambda x: (list(permutations(x,2)))).flatMap(lambda x: x).map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y).filter(lambda x: len(x[0]) != 0).filter(lambda x: x[1]>= threshold)
    pmi_intermediate_step = coOccurences.map(lambda x: ((x[0][0], x[0][1]), x[1], getTokenOccurence(x[0][0]), getTokenOccurence(x[0][1])))
    pmi = pmi_intermediate_step.map(lambda x: (x[0], log((((x[1])/(x[2]*x[3]))*totalLines),10) ,x[1], x[2], x[3]))
    
    return pmi.collect()

print(PMI(1000))


[(('that', 'the'), 0.06945860440719692, 2461, 10569, 24300), (('the', 'that'), 0.06945860440719692, 2461, 24300, 10569), (('to', 'the'), 0.05123525982989819, 4072, 18237, 24300), (('to', 'and'), 0.017522588038697152, 3815, 18237, 24604), (('the', 'to'), 0.05123525982989819, 4072, 24300, 18237), (('the', 'and'), 0.0459349918330654, 5427, 24300, 24604), (('and', 'to'), 0.017522588038697152, 3815, 24604, 18237), (('and', 'the'), 0.0459349918330654, 5427, 24604, 24300), (('the', 'be'), -0.09797128011614716, 1055, 24300, 6662), (('be', 'the'), -0.09797128011614716, 1055, 6662, 24300), (('all', 'the'), 0.14105230407117497, 1050, 3824, 24300), (('the', 'all'), 0.14105230407117497, 1050, 24300, 3824), (('all', 'and'), 0.15821868721651836, 1106, 3824, 24604), (('and', 'all'), 0.15821868721651836, 1106, 24604, 3824), (('this', 'of'), 0.1517481996162979, 1237, 6425, 16624), (('of', 'this'), 0.1517481996162979, 1237, 16624, 6425), (('my', 'and'), 0.0056896916303759305, 2351, 11549, 24604), (('and'

---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```

Hint: Sampling must be done at the very last step. Hint: there is an action that returns a sample subset from an RDD.


In [ ]:
from simple_tokenize import simple_tokenize
from math import log
import pprint

#Global variables to broadcast occurences
lines = sc.textFile("Shakespeare.txt")
occurence = lines.flatMap(lambda line: set(simple_tokenize(line))).map(lambda word: (word.lower().strip(), 1)).reduceByKey(lambda x,y: x+y).collectAsMap()
broadcastedOcc = sc.broadcast(occurence)

#Helper function to return token occurences using broadcasted variable
def getTokenOccurence(token):
  return broadcastedOcc.value[token]

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    lines = sc.textFile("Shakespeare.txt")
    totalLines = lines.count()
    coOccurences = lines.map(lambda x: set(simple_tokenize(x))).map(lambda x: (list(permutations(x,2)))).flatMap(lambda x: x).map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y).filter(lambda x: len(x[0]) != 0).filter(lambda x: x[1]>= threshold)
    pmi_intermediate_step = coOccurences.map(lambda x: ((x[0][0], x[0][1]), x[1], getTokenOccurence(x[0][0]), getTokenOccurence(x[0][1])))
    pmi = pmi_intermediate_step.map(lambda x: (x[0], log((((x[1])/(x[2]*x[3]))*totalLines),10) ,x[1], x[2], x[3]))
    returnlist = pmi.map(lambda x: (x[0][0], list(x))).groupByKey().map(lambda x: (x[0], list(x[1]))).takeSample(False, samp_size)
    
    return returnlist

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(PMI_one_token(2000,10))


[   ('will', [[('will', 'i'), 0.456978919724415, 2108, 4831, 18657]]),
    ('not', [[('not', 'i'), 0.2812323291754297, 2396, 8230, 18657]]),
    (   'and',
        [   [('and', 'to'), 0.017522588038697152, 3815, 24604, 18237],
            [('and', 'the'), 0.0459349918330654, 5427, 24604, 24300],
            [('and', 'my'), 0.0056896916303759305, 2351, 24604, 11549],
            [('and', 'you'), -0.05963472348492286, 2136, 24604, 12196],
            [('and', 'a'), 0.0006198226234107661, 2672, 24604, 13280],
            [('and', 'in'), 0.040318217968677454, 2340, 24604, 10614],
            [('and', 'of'), 0.028305447826683594, 3565, 24604, 16624],
            [('and', 'i'), -0.0503740353380564, 3338, 24604, 18657]]),
    (   'the',
        [   [('the', 'that'), 0.06945860440719692, 2461, 24300, 10569],
            [('the', 'to'), 0.05123525982989819, 4072, 24300, 18237],
            [('the', 'and'), 0.0459349918330654, 5427, 24300, 24604],
            [('the', 'of'), 0.34294075191889306,

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.